# Conversation
Now that we've trained and saved two indepedent models with two independent training sets, we want them to talk to each other. In a first step autoencoder1 produces a new song and sends it to autoencoder2 in PCA format. Autoencoder1 plays the audio, while Autoencoder2 analyze it and finds a 2D representation in its latent space. When Autoencoder1 is finished, Autoencoder2 tries to reproduce the song by decoding its latent space representation back to PCA. The process then starts to anew with switched roles.

In [1]:
from SoundGenerator import SoundGenerator
from Variational_Autoencoder_alla_Valerio import VAE
from SoundGenerator import SoundGenerator

In [3]:
WIN_LENGTH = int(44100 / 8)
HOP_LENGTH = int(44100 / 23)
N_FFT = int(44100 / 8)

First we need to load the two autoencoders and two corresponding SoundGenerators.

In [ ]:
autoencoder1_name="VAE_2D_1.0_24_2500samples"
autoencoder2_name="VAE_2D_1.0_24_2500samples"

valerio = VAE.load(autoencodeSoundGeneratorio_sg = SoundGenerator(
dennis = VAE.load(autoencoder2_name)

In [ ]:

        
    def sing